In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from imblearn.over_sampling import ADASYN
from collections import Counter
from scipy.spatial.distance import cdist
import seaborn as sa
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\ramje\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\ramje\anaconda3\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\ramje\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\ramje\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\ramje\anaconda3\lib\site-pack

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\ramje\anaconda3\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\ramje\anaconda3\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\ramje\anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\ramje\anaconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
    app.start()
  File "C:\Users\ramje\anaconda3\lib\site-pack

AttributeError: _ARRAY_API not found

In [2]:
# Step 2: Load the Dataset
df = pd.read_csv('churn-bigml.csv')

In [3]:
# Step 3: Inspect the Dataset
print("First 5 rows of the dataset:")
print(df.head())

First 5 rows of the dataset:
  State  Account length  Area code International plan Voice mail plan  \
0    KS             128        415                 No             Yes   
1    OH             107        415                 No             Yes   
2    NJ             137        415                 No              No   
3    OH              84        408                Yes              No   
4    OK              75        415                Yes              No   

   Number vmail messages  Total day minutes  Total day calls  \
0                     25              265.1              110   
1                     26              161.6              123   
2                      0              243.4              114   
3                      0              299.4               71   
4                      0              166.7              113   

   Total day charge  Total eve minutes  Total eve calls  Total eve charge  \
0             45.07              197.4               99             16

In [4]:
print("\nColumn Names:")
print(df.columns)


Column Names:
Index(['State', 'Account length', 'Area code', 'International plan',
       'Voice mail plan', 'Number vmail messages', 'Total day minutes',
       'Total day calls', 'Total day charge', 'Total eve minutes',
       'Total eve calls', 'Total eve charge', 'Total night minutes',
       'Total night calls', 'Total night charge', 'Total intl minutes',
       'Total intl calls', 'Total intl charge', 'Customer service calls',
       'Churn'],
      dtype='object')


In [5]:
# Step 4: Identify the Target Column ("Class")
if 'Churn' in df.columns:
    target_column = 'Churn'
else:
    raise ValueError("Class column not found! Please check the dataset.")

In [6]:
# Step 5: Encode categorical variables
label_encoders = {}
for column in df.columns:
    if df[column].dtype == 'object':
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
        label_encoders[column] = le

In [7]:
# Step 6: Split Data into Training & Testing Sets
X = df.drop(columns=['Churn'])
y = df['Churn']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# Step 7: Data Preprocessing (Standardization)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [9]:
# Step 8: Check Class Imbalance
print("\nClass Distribution Before ADASYN:")
print(Counter(y_train))


Class Distribution Before ADASYN:
Counter({False: 1823, True: 309})


In [10]:
# Step 9: Apply K-Means Clustering
k = 10  # Number of clusters
kmeans = KMeans(n_clusters=k, random_state=42)
clusters = kmeans.fit_predict(X_train)

In [11]:
print("\nClusters:", clusters)


Clusters: [3 0 6 ... 8 8 8]


In [12]:
# Step 10: Filter Clusters with More Minority Instances
df_train_clustered = X_train.copy()
df_train_clustered['cluster'] = clusters
df_train_clustered[target_column] = y_train

In [13]:
filtered_clusters = []
for c in np.unique(clusters):
    cluster_data = df_train_clustered[df_train_clustered['cluster'] == c]
    minority_count = cluster_data[target_column].sum()
    majority_count = len(cluster_data) - minority_count
    imbalance_ratio = (majority_count + 1) / (minority_count + 1)
    if imbalance_ratio < 1.5:
        filtered_clusters.append(c)

In [14]:
print("\nFiltered Clusters:", filtered_clusters)


Filtered Clusters: []


In [15]:
# Step 11: Compute Sampling Weight for Each Cluster
def compute_sampling_weight(cluster_data, target_column, minority_count, X_train):
    feature_data = cluster_data.drop(columns=['cluster', target_column]).values
    
    # Compute pairwise Euclidean distances
    pairwise_distances = cdist(feature_data, feature_data, metric='euclidean')
    
    # Take the mean of the upper triangle (excluding diagonal)
    avg_minority_dist = np.mean(pairwise_distances[np.triu_indices(len(feature_data), k=1)])
    
    print(f"Cluster {cluster_data['cluster'].iloc[0]} - Avg Euclidean Distance:", avg_minority_dist)
    
    density_factor = minority_count / (avg_minority_dist ** X_train.shape[1])
    sparsity_factor = 1 / density_factor
    
    return sparsity_factor

In [16]:
sparsity_sum = sum(compute_sampling_weight(
    df_train_clustered[df_train_clustered['cluster'] == f], 
    target_column, 
    len(df_train_clustered[df_train_clustered['cluster'] == f]), 
    X_train
) for f in filtered_clusters)

In [17]:
sampling_weights = {
    f: compute_sampling_weight(df_train_clustered[df_train_clustered['cluster'] == f], target_column, len(df_train_clustered[df_train_clustered['cluster'] == f]), X_train) / sparsity_sum
    for f in filtered_clusters
}

In [18]:
print("\nSampling Weights:", sampling_weights)


Sampling Weights: {}


In [19]:
# Step 12: Apply ADASYN for Oversampling
X_resampled = pd.DataFrame()
y_resampled = pd.Series(dtype=int)

In [20]:
for cluster_id in filtered_clusters:
    cluster_data = df_train_clustered[df_train_clustered['cluster'] == cluster_id]
    X_cluster = cluster_data.drop(columns=['cluster', target_column])
    y_cluster = cluster_data[target_column]
    
    num_samples = int(len(y_train) * sampling_weights[cluster_id])
    
    if len(y_cluster) < 2:  # Skip clusters with fewer than 2 samples
        continue
    
    try:
        adasyn = ADASYN(n_neighbors=min(len(y_cluster) - 1, 3), random_state=42)
        X_cluster_resampled, y_cluster_resampled = adasyn.fit_resample(X_cluster, y_cluster)
        X_resampled = pd.concat([X_resampled, X_cluster_resampled])
        y_resampled = pd.concat([y_resampled, y_cluster_resampled])
    except ValueError:
        print(f"Skipping cluster {cluster_id} due to ADASYN resampling issues.")

In [21]:
# Step 13: Check Class Distribution After ADASYN
print("\nClass Distribution After ADASYN:")
print(Counter(y_resampled))


Class Distribution After ADASYN:
Counter()


In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from imblearn.over_sampling import ADASYN  # If using SMOTE
import numpy as np

# Ensure X_train and y_train are defined before resampling
print("Before resampling:", X_train.shape, y_train.shape)

# Apply resampling (modify based on your technique)
adasyn = ADASYN(random_state=42)
X_resampled, y_resampled = adasyn.fit_resample(X_train, y_train)

print("After resampling:", X_resampled.shape, y_resampled.shape)  # Check shapes

# Step 13: Train a Classification Model
clf = RandomForestClassifier(random_state=42)
clf.fit(X_resampled, y_resampled)  # Train on resampled data

# Step 14: Make Predictions
y_pred = clf.predict(X_test)  # Predicted labels
y_proba = clf.predict_proba(X_test)[:, 1]  # Probability of positive class

# Step 15: Compute Accuracy and ROC AUC Score
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_proba)

print(f"\nAccuracy: {accuracy:.4f}")
print(f"ROC AUC Score: {roc_auc:.4f}")


Before resampling: (2132, 19) (2132,)
After resampling: (3625, 19) (3625,)

Accuracy: 0.9045
ROC AUC Score: 0.8878
